<a href="https://colab.research.google.com/github/DataSpott/SGT-Analysis/blob/master/SGT_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Initialization
import argparse
parser = argparse.ArgumentParser(description = 'Analyses growth curves for the SGT-method.')

parser.add_argument('checkerboard_nr', type = int, help = "Number of Checkerboards at the well-plate."
parser.add_argument('first_well', type = list, help = "First wells of all checkerboards on the plate.")
parser.add_argument('last_well', type = list, help = "Last wells of all Checkerboards on the plate.")
parser.add_argument('log_time', type = float, help = "Time in [min] that the investigated bacteria needs tto grow one log-level.")
parser.add_argument('antibiotic_one_name', type = str, help = "Name of the first antibiotic.", default = 'Antibiotic 1')
parser.add_argument('antibiotic_one_conc', type = list, help = "Concentrations of the first antibiotic.")
parser.add_argument('antibiotic_two_name', type = str, help = "Name of the second antibiotic.", default = 'Antibiotic 2')
parser.add_argument('antibiotic_two_conc', type = list, help = "Concentrations of the second antibiotic.")

arg = parser.parse_args()
checkerboard_nr = arg.checkerboard_nr
first_wells = arg.first_well
last_wells = arg.last_wells
log_time = arg.log_time
antibiotic_one_name = arg.antibiotic_one_name
antibiotic_one_conc = arg.antibiotic_one_conc
antibiotic_two_name = arg.antibiotic_two_name
antibiotic_two_conc = arg.antibiotic_two_conc

if int(len(first_wells)) > checkerboard_nr:
    print('To many first wells for checkerboards on plate.')
elif int(len(first_wells)) < checkerboard_nr:
    print('To less first wells for checkerboards on plate.')

if int(len(last_wells)) > checkerboard_nr:
    print('To many last wells for checkerboards on plate.')
elif int(len(last_wells)) < checkerboard_nr:
    print('To less last wells for checkerboards on plate.')

checkerboards = []

for board in range(checkerboard_nr):
    checkerboards.append([board, first_wells[board], last_wells[board]])


################################################################################
## Module-import

import pandas as pd
import numpy as np
import altair as alt
import xlrd as rd
import string
import subprocess as sub
import subprocess
import os
import sys

def install(package):
    sub.check_call([sys.executable, "-m", "pip", "install", package])

install('altair_saver')
install('selenium')

sub.check_call(['apt-get', 'install', '-y', 'chromium-chromedriver'],
     stdout=open(os.devnull,'wb'), stderr=None) 


import altair_saver

from scipy.optimize import curve_fit


################################################################################
## File-conversion

data_xls = pd.read_excel(fn, sheet_name=0)
data_xls.to_csv('data.csv', encoding = 'utf-8')

################################################################################
## Parsing and conversion to dataframe

#check if first cell of uploaded file is empty:
if pd.notna(data_xls.iloc[0, 0]):
    
    #if TRUE parsing of Tecan-format starts:
    #creates lists of possible well-names:
    letter_list = list(string.ascii_uppercase)
    number_list = np.linspace(1, 50, 50)
    possible_wells = []

    for letter in letter_list:
        for number in number_list:
        
            possible_well = letter + str(int(number))
            possible_wells.append(possible_well)
    
    first_column_name = data_xls.columns[0]

    well_index_list = []
    
    #check for possible-wells in first column of file:
    for well in possible_wells:
    
       index = data_xls.loc[data_xls[first_column_name] == well, [first_column_name]].index.tolist()
    
       if len(index) > 0:
        
           well_index_list.append([well, index])

    time_list = []
    get_time_list = []
 
    #get time-values from file:
    get_time_list.extend(data_xls.iloc[(int(well_index_list[0][1][0]) + 1), 1:])

    time_list.append(['Well', get_time_list])

    well_values = []

    #for-loop over all wells found in the file:
    for well in range(len(well_index_list)):
    
        well_name = well_index_list[well][0]
        
        #checks if well-number is < 10:
        if int(well_name[1:]) < 10:
            
            #if TRUE a 0 is added between letter and number:
            well_name = well_name[0] + '0' + well_name[1:]
    
        well_values.append([well_name, list(data_xls.iloc[(int(well_index_list[well][1][0]) + 3), 1:].values)])

    time_list.extend(well_values)

    raw_data_dict = {}

    #for-loop over all times in the extended time_list:
    for times in range(len(time_list)):
    
        #sets the well-name (or in case of the time-data the string 'Well') as keys for their values in the dicitionary:
        raw_data_dict[str(time_list[times][0])] = time_list[times][1]

    #creates dataframe 'data_raw' with the dictionary:
    data_raw = pd.DataFrame(raw_data_dict)

    #transposes the dataframe:
    data_raw = data_raw.transpose()

    #sets the index to the first column:
    data_raw.reset_index(level=0, inplace=True)

    #sets the first row as column-names:
    data_raw.columns = data_raw.iloc[0]

    #deletes the first row
    data_raw = data_raw[1:]

    #displays dataframe 'data_raw':
    #print(data_raw)#[data_raw['Well'] == input("Choose well by entering the well-name (e.g.: 'A01'): ")] #Optional: only a User-chosen Well is displayed


elif pd.isna(data_xls.iloc[0, 0]):
    #Build a pandas dataframe from the .csv-file:
    data_raw = pd.read_csv('data.csv')

    #deletes rows with unnecessary information (temp.-values & measurement-data):
    data_raw = data_raw[:-5]
    data_raw = data_raw.iloc[1:]

    #brings the dataframe in an optimised form for plotting:
    data_raw = data_raw.rename(columns={'Unnamed: 0.1': 'Well'})
    del data_raw['Unnamed: 0']

    #deletes the 's' from the time-values:
    #for-loop over all columns
    for index in range(len(data_raw.columns)):
        data_raw.columns.values[index] =  data_raw.columns.values[index].replace('s', '')

    #add a '0' for Wells with single digit numbers:
    #for-loop over the row-count of the dataframe:
    for index in range(len(data_raw)):
        
        #check if well-name is two signs long:
        if len(data_raw.iloc[index, 0]) == 2:
            
            #if TRUE a 0 is added between the letter and number:
            string = data_raw.iloc[index, 0]
            string = string[:1] + '0' + string[1:]
            data_raw.iloc[index, 0] = string

    #displays dataframe 'data_raw':
    #print(data_raw)#[data_raw['Well'] == input("Choose well by entering the well-name (e.g.: 'A01'): ")] #Optional: only a User-chosen Well is displayed


################################################################################
## Wide-to-long-conversion

#uses the panda melt-command to bring the dataframe from wide to long form, where Well is an 'ID#, by which specific values (Time & Value) are identified:
data_melted = pd.melt(data_raw, id_vars = ['Well'], var_name = 'Time', value_name = 'Value')

#changes data type of the columns 'Time' & 'Value':
data_melted['Time'] = data_melted['Time'].astype(float)
data_melted['Value'] = data_melted['Value'].astype(float)

#displays new dataframe 'data_melted':
#data_melted#[data_melted['Well'] == input("Choose well by entering the well-name (e.g.: 'A01'): ")] #Optional: only a User-chosen Well is displayed


################################################################################
## Sigmoid curve-fitting

#Definition of the sigmoid-function:
def sigmoid(x, L, x0, k, n):                                                    #L = curve's maximum value; x0 = x-value of the sigmoid´s midpoint (turnpoint, where the curve is linear); k = logistic growth rate or steepness of the curve (; n = value for y at x = 0)
    y = L / (1 + np.exp(-k * (x - x0))) + n
    return y

#creates list of all wells in dataframe 'data_merged':
well_plate = list(data_melted['Well'])
well_plate = list(dict.fromkeys(well_plate))

fitted_values = []
popt_list = []

#for-loop over all wells of the plate:
for well in well_plate:
    
    #initialize data:
    x_data = list(data_melted[data_melted['Well'] == well]['Time']) 
    y_data = list(data_melted[data_melted['Well'] == well]['Value']) 
    
    #initialize start-parameters for curve_fit:
    y_max = max(y_data)
    y_min = min(y_data)
    k = (y_max/1000)          
    half_index = int(len(x_data)/2)
    x0 = data_melted[data_melted['Well'] == well].iloc[half_index, 1]                   
    
    #initialize parameters for the sigmoid-function (= L, x0, k, n)
    p0 = [y_max, x0, k, y_min]
    
    #curve_fit:
    popt, pcov = curve_fit(sigmoid, x_data, y_data, p0)
    
    popt_list.append([well, popt[0], popt[1], popt[2], popt[3]])

    #get fitted function from optimized parameters:
    fitted_y = sigmoid(x_data, *popt)
    
    fitted_values.append([well, np.array(x_data), fitted_y])
    
#displays list 'fitted_values':
#print(fitted_values)


################################################################################
## Computation of fitted data

fitted_data = []

#primary for-loop over all fitted values:
for wells in range(len(fitted_values)):
    
    #secondary for-loop over time-values for the actual well: 
    for position in range(len(fitted_values[wells][1])):
        
        #get well-name, time & fitted value
        well = fitted_values[wells][0]
        time = fitted_values[wells][1][position]                                        
        fitted_y = fitted_values[wells][2][position]
        
        fitted_data.append([well, time, fitted_y])

#creation of a second dataframe with the columns 'Well', 'Time' and 'Value' from the list 'fitted_data':
data_fitted = pd.DataFrame(fitted_data, columns = ['Well', 'Time', 'Fitted Value'])

#displays dataframe 'data_fitted':
#print(data_fitted)#[data_fitted['Well'] == input("Choose well by entering the well-name (e.g.: 'A01'): ")] #Optional: only a User-chosen Well is displayed


################################################################################
## Dataframe-merging

#create multiindex 'Well' and 'Time' for dataframe 'data_fitted':
data_fitted_multiindex = data_fitted.set_index(['Well', 'Time'], drop = True)

#check if dataframe 'data_merged' exist:
try:
    data_merged
except NameError:
    data_merged_exists = False
else:
    data_merged_exists = True

#if 'data_merged' exists:
if data_merged_exists == True: 
    
    #check if dataframe 'data_merged' already a has a column 'µ_max':
    if hasattr(data_merged, 'Fitted Value'):
    
        #if TRUE delete the column and merge 'data_µ_max' into 'data_merged':
        del data_merged['Fitted Value']
        data_merged = data_melted.join(data_fitted_multiindex, on = ['Well', 'Time'])

#if 'data_merged' doesn´t exist join 'data_µ_max' into 'data_merged':
else:
    data_merged = data_melted.join(data_fitted_multiindex, on = ['Well', 'Time'])

#display dataframe 'data_merged':
#print(data_merged)#[data_merged['Well'] == input("Choose well by entering the well-name (e.g.: 'A01'): ")] #Optional: only a User-chosen Well is displayed


################################################################################
## Plotting of the raw data and fitted sigmoid-curves

#get a list of all wells in dataframe 'data':
plate_wells = list(data_merged['Well'])

well_numbers = []

#for-loop over all wells in dataframe 'data_merged':
for wells in range(len(plate_wells)):
    
    #gets the number of the actual well and appends it to the array:
    well_numbers.append(plate_wells[wells][1:])

#deletes duplicates in the array:
well_numbers = list(dict.fromkeys(well_numbers))

#sets the number of rows in which graphs are plotted:
plot_columns = int(max(well_numbers))

#deactivates the max_row limit of altair for the processed dataframe:
alt.data_transformers.enable('default', max_rows=None)

#raw-data-plot:
raw_data = alt.Chart(data_merged).mark_point(opacity=0.4, color = "teal"

#columns to plot with specifications for the axis:
).encode(
    alt.X('Time:Q', scale=alt.Scale(domain=(0, 66000)), axis=alt.Axis(title='Time [s]')),
    alt.Y('Value:Q', axis=alt.Axis(title='OD (600 nm)'))

#properties of the resulting diagramm:
).properties(
    width = 300,
    height = 200
)

#fitted-data-plot:
fitted_curves = alt.Chart(data_merged).mark_line(color = "black"
).encode(
    alt.X('Time:Q'),
    alt.Y('Fitted Value:Q')
 )

#command to layer the raw- and fitted-data-plots over each other and split them into many diagramms depending on the well:
alt.layer(raw_data, fitted_curves, data = data_merged).facet(facet = 'Well', columns = plot_columns)


################################################################################
## Cut-off-determination

cut_off = float(input("Set CutOff: "))


################################################################################
## µ-calculation & tangent-application

#variables matching the possible user-input for comparison in the if-condition:
decision_positive = 'Yes'
decision_negative = 'No'

#user-decision to progress either manually or automatic:
print("This program offers you the possibility to determine the tangent for the linear area of the growth curves either manually or automatic.")
decision_manual_automatic = input("Do you want to manually set an area for the tangent-calculation? Enter Yes/No: ")

#function to find value in a given array that is closest to a given value:
def find_nearest(array, value):                                                 
    
    #converts the given list/array/... to an array:
    array = np.asarray(array)
    
    #finds the index of the array-value, which results in the lowest absolute value by subtraction with the searched value:
    idx = (np.abs(array - value)).argmin()
    
    #returns the array-value:
    return array[idx]

#check if the user want to manually perform the tangent-fitting:
if decision_manual_automatic.lower() == decision_positive.lower():
    
    #User-input of the upper and lower boundaries for the linear curve area:
    upper_boundary = float(input("Enter the upper boundary for the linear growth area of the curves (e.g. 0.6): "))
    print('\n')
    lower_boundary = float(input("Enter the lower boundary for the linear growth area of the curves (e.g. 0.4): "))

    #Well-list:
    well_plate = list(data_merged['Well'])
    well_plate = list(dict.fromkeys(well_plate))

    boundary_values = []
    
    #for-loop over all wells:
    for well in well_plate:
        
        #variable that references the 'Fitted Value'-column of dataframe data_merged:
        fitted_value_column = data_merged[data_merged['Well'] == well]['Fitted Value']
        
        #searches the nearest fitted values to the user-specified boundaries for the actual well:
        nearest_upper_boundary = find_nearest(fitted_value_column, upper_boundary)
        nearest_lower_boundary = find_nearest(fitted_value_column, lower_boundary)
        
        #searches the corresponding times to the boundary-closest fitted values:
        upper_time = data_merged[(data_merged['Well'] == well) & (data_merged['Fitted Value'] == nearest_upper_boundary)]['Time'].values[0]
        lower_time = data_merged[(data_merged['Well'] == well) & (data_merged['Fitted Value'] == nearest_lower_boundary)]['Time'].values[0]
        
        #calculates the deltas for the boundary-values:
        delta_y = (upper_boundary - lower_boundary)
        delta_x = (upper_time - lower_time)
        
        #checks if division by 0 occured:
        if delta_x > 0 or delta_x < 0:
            
            #if False µ is calculated:
            µ = (delta_y / delta_x)
        
        else:
            µ = 'division by 0'
        
        boundary_values.append([well, upper_boundary, upper_time, lower_boundary, lower_time, µ])
    
    y0_list = []
    
    #for-loop over all wells in the second main-array:
    for well in range(len(boundary_values)):
        
        if type(boundary_values[well][5]) != str:
            
            #setup initial data:
            m = float(boundary_values[well][5])
            x = float(boundary_values[well][2])
            y = float(boundary_values[well][1])
    
            #compute n from y = mx + n:
            n = (y - (m * x))

            y0_list.append([boundary_values[well][0], m, n])
        
        else:
            
            m = boundary_values[well][5]
            n = 'not computable'
            y0_list.append([boundary_values[well][0], m, n])
    
    #create dataframe from the resulting array:
    data = pd.DataFrame(y0_list, columns = ['Well', 'µ_max', 'y0'])
    
    #display resulting dataframe:
    #print(data)

#
#
#check if the user want to autmatically perform the tangent-fitting:
if decision_manual_automatic.lower() == decision_negative.lower():
    
    #Well-list:
    well_plate = list(data_merged['Well'])
    well_plate = list(dict.fromkeys(well_plate))

    µ_values = []

    #primary for-loop over all wells:
    for well in well_plate:
    
        #row-count for that well as condition for secondary for-loop:
        row_count = data_merged[data_merged['Well'] == well].shape[0]
    
        µ = []
        time = []

        #secondary loop over all rows for that well:
        for row in range(1, row_count):
        
            #initialize data:
            actual_value = data_merged[data_merged['Well'] == well].iloc[(row - 1), 3]
            actual_time = data_merged[data_merged['Well'] == well].iloc[(row - 1), 1]
            next_value = data_merged[data_merged['Well'] == well].iloc[row, 3]
            next_time = data_merged[data_merged['Well'] == well].iloc[row, 1]
        
            #process data:
            delta_x = next_time - actual_time
            delta_y = next_value - actual_value
            m = (delta_y / delta_x)
            
            µ.append(m)
            time.append(next_time)
    
        µ_values.append([well, µ, time])

    #display resulting array:
    #print(µ_values)

    #define function to find max µ-values in array:
    def µ_max(array):
    
        µ_max_list = []
    
        #primary for-loop over all wells:
        for wells in range(len(array)):
        
            #setup initial data:
            well = array[wells][0]
            µ_max = array[wells][1][0]
            µ_max_time = array[wells][2][0]
        
            #secondary for-loop over all µ-values for that well:
            for value in range(1, len(array[wells][1])):
            
                #if-condition to compare µ-values and find max µ:
                if array[wells][1][value] >= array[wells][1][(value - 1)]:
                
                    #set new µ_max and corresponding time:
                    µ_max = array[wells][1][value]
                    µ_max_time = array[wells][2][value]
        
            µ_max_list.append([well, µ_max_time, µ_max])
        
        return µ_max_list

    #start function µ_max with the µ_values-array:
    µ_max_list = µ_max(µ_values)

    #set Pandas display-precision to show 3 decimal places:
    pd.set_option('display.precision', 3)

    #create a dataframe from the resulting array:
    data_µ_max = pd.DataFrame(µ_max_list, columns = ['Well', 'Time', 'µ_max'])

    #display resulting dataframe:
    #print("data_µ_max: ", data_µ_max, "\n")#[data_µ_max['Well'] == input("Choose well by entering the well-name (e.g.: 'A01'): ")] #Optional: only a User-chosen Well is displayed


################################################################################
## Dataframe-merging for automated µ-calculation

    #create multiindex 'Well' and 'Time' for dataframe 'data_µ_max':
    data_µ_max_multiindex = data_µ_max.set_index(['Well', 'Time'], drop = True)

    #check if dataframe 'data_merged' already a has a column 'µ_max':
    if hasattr(data_merged, 'µ_max'):
    
        #if TRUE delete the column and merge 'data_µ_max' into 'data_merged':
        del data_merged['µ_max']
        data_merged = data_merged.join(data_µ_max_multiindex, on = ['Well', 'Time'])

    #if FALSE merge 'data_µ_max' into 'data_merged':
    else:
        data_merged = data_merged.join(data_µ_max_multiindex, on = ['Well', 'Time'])

    #display dataframe 'data_merged':
    #print("data_merged:\n", data_merged, "\n")#[data_merged['Well'] == input("Choose well by entering the well-name (e.g.: 'A01'): ")] #Optional: only a User-chosen Well is displayed


################################################################################
## Creation of dataframe 'data'

    y0_list = []

    #for-loop over all wells:
    for wells in well_plate:
    
        #setup initial data:
        m = data_merged[(data_merged['Well'] == wells) & (pd.notna(data_merged['µ_max']))]['µ_max'].values[0]
        x = data_merged[(data_merged['Well'] == wells) & (pd.notna(data_merged['µ_max']))]['Time'].values[0]
        y = data_merged[(data_merged['Well'] == wells) & (pd.notna(data_merged['µ_max']))]['Fitted Value'].values[0]
    
        #compute n from y = mx + n
        n = y - m * x
    
        y0_list.append([wells, m, n])

    #create dataframe from the resulting array:
    data = pd.DataFrame(y0_list, columns = ['Well', 'µ_max', 'y0'])

    #display dataframe 'data':
    #print("data:\n", data)#[data['Well'] == input("Choose well by entering the well-name (e.g.: 'A01'):")] #Optional: only a User-chosen Well is displayed


################################################################################
## SGT-calculation

#row-count as condititon for following loop:
row_count = data.shape[0]

#for-loop over all rows of 'data':
for row in range(row_count):
    
    #setup initial data:
    µ_max = data.loc[row, 'µ_max']
    y0 = data.loc[row, 'y0']
    
    #check if µ_max is no string:
    if type(data.loc[row, 'µ_max']) != str:
        
        #compute SGT and add it to 'data':
        data.loc[row, 'SGT'] = int(((cut_off - y0) / µ_max) / 60)
    
    else:

        data.loc[row, 'SGT'] = 'not computable'

#displays dataframe 'data':
#data#[data['Well'] == input("Choose well by entering the well-name (e.g.: 'A01'): ")] #Optional: only a User-chosen Well is displayed

################################################################################
## log-level-reduction calculation

#get row-count of dataframe 'data':
row_count = data.shape[0]

calculated_values = []

#for-loop over all checkerboards:
for boards in range(len(checkerboards)):
    
    #initialize data for if-conditions:
    first_well = str(checkerboards[boards][1])
    last_well = str(checkerboards[boards][2])
    
    #for-loop over all wells in dataframe 'data':
    for rows in range(row_count):
        
        #get name of the specific well:
        well_name = str(data.iat[rows, 0])

        #check if well is in the limits of the checkerboard:
        if well_name[0] >= first_well[0] and well_name[0] <= last_well[0]:
            if int(well_name[1:]) >= int(first_well[1:]) and int(well_name[1:]) <= int(last_well[1:]):
                
                if type(data.loc[rows,'SGT']) != str:
                    
                    #calculate data:
                    SGT_subtracted_value = int(data[data['Well'] == well_name]['SGT'].values[0] 
                                          - data[data['Well'] == first_well]['SGT'].values[0])
                    log_level_reduction = round((SGT_subtracted_value / log_time), 1)
                
                    #determine antibiotic-concentrations:
                    first_antibiotic_index = (ord(well_name[0]) - ord(first_well[0])) + 1
                    first_antibiotic_concentration = float(antibiotic_one[first_antibiotic_index])
                    second_antibiotic_index = int(int(well_name[1:]) - ((12 / board_count) * boards))
                    second_antibiotic_concentration = float(antibiotic_two[second_antibiotic_index])

                    #append data to main-array:
                    calculated_values.append([well_name, SGT_subtracted_value, 
                        log_level_reduction, first_antibiotic_concentration, 
                        second_antibiotic_concentration])
                else:
                    print(well_name)
                    SGT_subtracted_value = 'not computable'
                    log_level_reduction = 'not computable'
                    first_antibiotic_index = (ord(well_name[0]) - ord(first_well[0])) + 1
                    first_antibiotic_concentration = float(antibiotic_one[first_antibiotic_index])
                    second_antibiotic_index = int(int(well_name[1:]) - ((12 / board_count) * boards))
                    second_antibiotic_concentration = float(antibiotic_two[second_antibiotic_index])
                    calculated_values.append([well_name, SGT_subtracted_value, 
                        log_level_reduction, first_antibiotic_concentration, 
                        second_antibiotic_concentration])
        
        #check if well-letter is out of the checkerboard-bounds:
        elif well_name[0] <= first_well[0] or well_name[0] >= last_well[0]:
            if int(well_name[1:]) >= int(first_well[1:]) and int(well_name[1:]) <= int(last_well[1:]):
                
                if type(data.loc[rows,'SGT']) != str:
                    
                    #calculate data:
                    SGT_subtracted_value = int(float(data[data['Well'] == well_name]['SGT'].values[0]) 
                                      - float(data[data['Well'] == first_well]['SGT'].values[0]))
                    log_level_reduction = round((SGT_subtracted_value / log_time), 1)
                
                    #determine antibiotic-concentrations:
                    first_antibiotic_concentration = '-'
                    second_antibiotic_concentration = '-'

                    #append data to main-array:
                    calculated_values.append([well_name, SGT_subtracted_value, 
                        log_level_reduction, first_antibiotic_concentration, 
                        second_antibiotic_concentration])
                
                else:
                    SGT_subtracted_value = 'not computable'
                    log_level_reduction = 'not computable'
                    first_antibiotic_concentration = '-'
                    second_antibiotic_concentration = '-'
                    calculated_values.append([well_name, SGT_subtracted_value, 
                        log_level_reduction, first_antibiotic_concentration, 
                        second_antibiotic_concentration])
        
        #check if well-number is out of the checkerboard-bounds:
        elif int(well_name[1:]) <= int(first_well[1:]) or int(well_name[1:]) >= int(last_well[1:]):
            if well_name[0] >= first_well[0] and well_name[0] <= last_well[0]:
                
                if type(data.loc[rows,'SGT']) != str:
                    
                    #calculate data:
                    SGT_subtracted_value = int(data[data['Well'] == well_name]['SGT'].values[0] 
                                      - data[data['Well'] == first_well]['SGT'].values[0])
                    log_level_reduction = round((SGT_subtracted_value / log_time), 1)
                
                    #determine antibiotic-concentrations:
                    first_antibiotic_concentration = '-'
                    second_antibiotic_concentration = '-'

                    #append data to main-array:
                    calculated_values.append([well_name, SGT_subtracted_value, 
                        log_level_reduction, first_antibiotic_concentration, 
                        second_antibiotic_concentration])

                else:
                    SGT_subtracted_value = 'not computable'
                    log_level_reduction = 'not computable'
                    first_antibiotic_concentration = '-'
                    second_antibiotic_concentration = '-'
                    calculated_values.append([well_name, SGT_subtracted_value, 
                        log_level_reduction, first_antibiotic_concentration, 
                        second_antibiotic_concentration])


#creates dataframe from calculated data:
data_calculated_values = pd.DataFrame(calculated_values, columns = ['Well',
    'SGT minus control', 'log-level-reduction',
    (antibiotic_one_name + '-concentration'),
    (antibiotic_two_name + '-concentration')])

#displays dataframe 'data_calculated_values':
#print(data_calculated_values)#[data_calculated_values['Well'] == input("Choose well by entering the well-name (e.g.: 'A01'): ")] #Optional: only a User-chosen Well is displayed


################################################################################
## Dataframe-merging

#set column 'Well' as index:
data_calculated_values_index = data_calculated_values.set_index(['Well'], drop = True)

antibiotic_one_column = str(antibiotic_one_name + '-concentration')
antibiotic_two_column = str(antibiotic_two_name + '-concentration')

#check if dataframe 'data' already has the columns from dataframe 'data_calculated_values'
#if TRUE delete the columns and merge 'data_calculated_values' into 'data':
if hasattr(data, 'SGT minus control'):
    del data['SGT minus control']
    
    if hasattr(data, 'log-level-reduction'):
        del data['log-level-reduction']
        
        if hasattr(data, antibiotic_one_column):
            del data[antibiotic_one_column]
            
            if hasattr(data, antibiotic_two_column):
                del data[antibiotic_two_column]
                data = data.merge(data_calculated_values_index, on = ['Well'])
            
            else:
                data = data.merge(data_calculated_values_index, on = ['Well'])
        else:
            data = data.merge(data_calculated_values_index, on = ['Well'])
    else:
        data = data.merge(data_calculated_values_index, on = ['Well'])

#if FALSE merge 'data_calculated_values' into 'data':
else:
    data = data.merge(data_calculated_values_index, on = ['Well'])

#display dataframe 'data':
#data#[data['Well'] == input("Choose well by entering the well-name (e.g.: 'A01'): ")] #Optional: only a User-chosen Well is displayed


################################################################################
## Determination of BBC-Wells

#get row-count of dataframe 'data':
row_count = data.shape[0]

first_antibiotic_bbcs = []
second_antibiotic_bbcs = []

#for-loop over all checkerboards:
for boards in range(len(checkerboards)):
    
    first_antibiotic_concentrations =[]

    #initialize data for if-conditions:
    first_well = str(checkerboards[boards][1])
    last_well = str(checkerboards[boards][2])
        
    #for-loop over all wells in dataframe 'data':
    for rows in range(row_count):
        
        #get name of the specific well:
        well_name = str(data.iat[rows, 0])
        
        #check if well is within the limits of the checkerboard and accomplishs the log-level-reduction:
        if well_name[1:] == first_well[1:]:
            if well_name[0] >= first_well[0] and well_name[0] <= last_well[0]:
                if data[data['Well'] == well_name]['log-level-reduction'].values[0] >= 3.0:
                    
                    #get concentration of the first antibiotic in this well:
                    first_antibiotic_concentrations.append(data.loc[rows, antibiotic_one_column])

    first_antibiotic_bbcs.append(first_antibiotic_concentrations)

#for-loop over all checkerboards:
for boards in range(len(checkerboards)):
    
    second_antibiotic_concentrations = []

    #initialize data for if-conditions:
    first_well = str(checkerboards[boards][1])
    last_well = str(checkerboards[boards][2])
        
    #for-loop over all wells in dataframe 'data':
    for rows in range(row_count):
        
        #get name of the specific well:
        well_name = str(data.iat[rows, 0])
        
        #check if well is within the limits of the checkerboard and accomplishs the log-level-reduction:
        if well_name[0] == first_well[0]:
            if well_name[1:] >= first_well[1:] and well_name[1:] <= last_well[1:]:
                if data[data['Well'] == well_name]['log-level-reduction'].values[0] >= 3.0:
                    
                    #get concentration of the second antibiotic in this well:
                    second_antibiotic_concentrations.append(data.loc[rows, antibiotic_two_column])

    second_antibiotic_bbcs.append(second_antibiotic_concentrations)

#print(first_antibiotic_bbcs, second_antibiotic_bbcs)

first_antibiotic_bbc = []
second_antibiotic_bbc = []

#for-loop over all checkerboards:
for boards in range(len(checkerboards)):
    
    #append corresponding data from first main-arrays to new main-arrays:
    first_antibiotic_bbc.append(first_antibiotic_bbcs[boards][0])
    second_antibiotic_bbc.append(second_antibiotic_bbcs[boards][0])

#print(first_antibiotic_bbc, second_antibiotic_bbc)


################################################################################
## Determination of synergism

row_count = data.shape[0]

fici_values = []

#for-loop over all checkerboards:
for boards in range(len(checkerboards)):
    
    #initialize data for if-conditions:
    first_well = str(checkerboards[boards][1])
    last_well = str(checkerboards[boards][2])
    
    #initialize data for computation:
    antibiotic_one_bbc = first_antibiotic_bbc[boards]
    antibiotic_two_bbc = second_antibiotic_bbc[boards]
    
    fici_value_per_well = []
    wells = []

    #for-loop over all wells in dataframe 'data':
    for rows in range(row_count):
        
        #get name of the specific well:
        well_name = str(data.iat[rows, 0])
        
        #check if well is in the limits of the checkerboarda and the log-level-reduction is >= 3:
        if well_name[0] > first_well[0] and well_name[0] <= last_well[0]:
            if int(well_name[1:]) > int(first_well[1:]) and int(well_name[1:]) <= int(last_well[1:]):
                if type(data.loc[rows, 'log-level-reduction']) != str:
                    if data[data['Well'] == well_name]['log-level-reduction'].values[0] >= 3.0:
                        actual_first_antibiotic_concentration = data[data['Well'] == well_name][antibiotic_one_column].values[0]
                        actual_second_antibiotic_concentration = data[data['Well'] == well_name][antibiotic_two_column].values[0]
                        well_fici_value = ((actual_first_antibiotic_concentration/antibiotic_one_bbc) + (actual_second_antibiotic_concentration/antibiotic_two_bbc))
                    
                        wells.append(str(well_name))
                        fici_value_per_well.append(well_fici_value)

    fici_values.append([wells, fici_value_per_well])

#print(fici_values)


################################################################################
## Identification of first wells in rows & columns

#defines a function which seeks the first letter in alphabetical order and the smallest number in an given list of well-names:
def well_naming(array):
    
    #sets the letter and number of the first well in the list:
    well_letter = array[0][0]
    well_number = array[0][1:]
    
    #tests for all wells of the list if the letter and number are smaller and if true sets them es new minimal value:
    for wells in range(len(array)):
        if array[wells][0] < well_letter:
            well_letter = array[wells][0]
        if array[wells][1:] < well_number:
            well_number = array[wells][1:]
    
    #returns the minimal letter and number of all wells:
    return well_letter, well_number

#get a list of all wells in dataframe 'data':
plate_wells = list(data['Well'])

well_letters = []
well_numbers = []
first_wells_in_rows_and_columns = []

#for-loop over all wells in dataframe 'data':
for wells in range(len(plate_wells)):
    
    #gets the letter and number of the actual well and appends them to the arrays:
    well_letters.append(plate_wells[wells][0])
    well_numbers.append(plate_wells[wells][1:])

#deletes duplicates in the arrays:
well_letters = list(dict.fromkeys(well_letters))
well_numbers = list(dict.fromkeys(well_numbers))

#for_loop over all checkerboards:
for boards in range(len(checkerboards)):
    
    first_well_in_row = []
    first_well_in_column = []
    
    #gets the wells with an log-level-reduction >= 3:
    wells = fici_values[boards][0]
    
    #searches the minimal letter and number of this wells:
    well_min_letter_and_number = well_naming(wells)
    
    #for-loop over all letters that are part of an well-name in dataframe 'data':
    for letters in range(len(well_letters)):
        
        #takes the actual letter and searches the well-list for all wells with the letter in their name:
        letter = well_letters[letters]
        well_selection = [s for s in wells if letter in s]
        
        #checks if wells which names contain the letter were found:
        if len(well_selection) > 0:
            
            #gets the "smallest" of the selected ones (the first for this row) and appends it to the array:
            first_well = min(well_selection)
            first_well_in_row.append(first_well)
            
            #checks if the array contains a well with the minimal number in it´s name:
            end_well_number = well_min_letter_and_number[1]
            if first_well_in_row[-1][1:] == end_well_number:   
                
                #if TRUE the loop is interrupted
                break
    
    #for-loop over all numbers that are part of an well-name in dataframe 'data':
    for numbers in range(len(well_numbers)):
        
        #takes the actual number and searches the well-list for all wells with the number in their name:
        number = well_numbers[numbers]
        well_selection = [s for s in wells if number in s]
        
        #checks if wells which names contain the number were found:
        if len(well_selection) > 0:
            
            #gets the "smallest" of the selected ones (the first for this column) and appends it to the array:
            first_well = min(well_selection)
            first_well_in_column.append(first_well)
            
            #checks if the array contains a well with the minimal letter in it´s name:
            end_well_letter = well_min_letter_and_number[0]
            if first_well_in_column[-1][0] == end_well_letter:   
                
                #if TRUE the loop is interrupted
                break
    
    #extend the row-array with the column-array:
    first_well_in_row.extend(first_well_in_column)
    
    #deletes all duplicates in the extended row-array and saves it as new variable:
    first_wells_in_rows_and_columns.append(list(dict.fromkeys(first_well_in_row)))

#print(first_wells_in_rows_and_columns)


################################################################################
## FICI-value-calculation for determined first wells

board_fici_values = []
board_fici_values_with_well = []

#for-loop over all checkerboards:
for boards in range(len(checkerboards)):
    
    corresponding_fici_value = []
    corresponding_fici_value_with_well = []
    
    #for-loop over all wells that are the first well of a row/column at that checkerboard:
    for wells in range(len(first_wells_in_rows_and_columns[boards])):
        
        #searches the index of the actual well in the fici-values-array:
        well_index = fici_values[boards][0].index(first_wells_in_rows_and_columns[boards][wells])
        
        #appends the fici_value which corresponds to this index to the arrays:
        corresponding_fici_value.append(fici_values[boards][1][well_index])
        corresponding_fici_value_with_well.append([first_wells_in_rows_and_columns[boards][wells], fici_values[boards][1][well_index]])
    
    board_fici_values.append(corresponding_fici_value)
    board_fici_values_with_well.append(corresponding_fici_value_with_well)

fici_average = []

#for-loop over all checkerboards:
for boards in range(len(checkerboards)):
    
    #computes the average of the fici-values of the first wells in all rows and columns of the checkerboard:
    fici_average.append(np.average(board_fici_values[boards]))

#displays the resulting arrays:
# for boards in range(len(checkerboards)):
#     print("First wells of all rows and columns of Checkerboard %s with an log-level-reduction >= 3 and their corresponding FICI-value:"
#     %(boards + 1), '\n',
#     board_fici_values_with_well[boards],
#     '\n',
#     "The average of the FICI-values for this board is: ",
#     fici_average[boards],
#     '\n')


################################################################################
## Plotting of complete graphs

data_results_plot = data_merged.copy()

time_values = data_results_plot.loc[:, 'Time']
time_values = list(dict.fromkeys(time_values))

well_list = data_results_plot.loc[:, 'Well']
well_list = list(dict.fromkeys(well_list))

tangent_values = []

for wells in well_list:
    actual_well = wells
    µ = data[data['Well'] == actual_well]['µ_max'].values[0]
    n = data[data['Well'] == wells]['y0'].values[0]
    µ_max_index = data_results_plot[(data_results_plot['Well'] == wells) & (pd.notna(data_results_plot['µ_max']))]['µ_max'].index.values[0]
    data_results_plot.loc[µ_max_index, 'µ_max corresponding Value'] = data_results_plot.loc[µ_max_index, 'Fitted Value']
    if type(µ) != str:  
        for time in time_values:
            y = ((time * µ) + n)
            tangent_values.append([wells, time, y])

for elements in tangent_values:
    row_index = data_results_plot[(data_results_plot['Well'] == elements[0]) & (data_results_plot['Time'] == elements[1])].index[0]
    data_results_plot.loc[row_index, 'Tangent y-value'] = elements[2]

if decision_manual_automatic.lower() == decision_positive.lower():
    data_results_plot.loc[:, 'Upper Boundary'] = upper_boundary
    data_results_plot.loc[:, 'Lower Boundary'] = lower_boundary

plate_wells = list(data_results_plot['Well'])

well_numbers = []


for wells in range(len(plate_wells)):
    
    well_numbers.append(plate_wells[wells][1:])

well_numbers = list(dict.fromkeys(well_numbers))
plot_columns = int(max(well_numbers))

alt.data_transformers.enable('default', max_rows=None)

raw_data = alt.Chart(data_results_plot).mark_point(opacity=0.4, color = "teal"
).encode(
    alt.X('Time:Q', scale=alt.Scale(domain=(0, 66000)), axis=alt.Axis(title='Time [s]')),
    alt.Y('Value:Q', scale=alt.Scale(domain=(0, 1.5)), axis=alt.Axis(title='OD (600 nm)'))
).properties(
    width = 300,
    height = 200
)

fitted_curves = alt.Chart(data_results_plot).mark_line(color = "black"
).encode(
    alt.X('Time:Q'),
    alt.Y('Fitted Value:Q')
)

tangent = alt.Chart(data_results_plot
).mark_line(color = "orange", opacity = 0.5
).encode(
    alt.X('Time:Q'),
    alt.Y('Tangent y-value:Q')
).transform_filter(
    alt.FieldRangePredicate(field='Tangent y-value', range=[0, 1.5]))

if decision_manual_automatic.lower() == decision_positive.lower():
    
    upper_border = alt.Chart(data_results_plot).mark_line(color = "red", opacity = 0.35
    ).encode(
        alt.X('Time:Q'),
        alt.Y('Upper Boundary:Q')
    )

    lower_border = alt.Chart(data_results_plot).mark_line(color = "red", opacity = 0.35
    ).encode(
        alt.X('Time:Q'),
        alt.Y('Lower Boundary:Q')
    )

    plotted = alt.layer(raw_data, fitted_curves, tangent, upper_border, lower_border, data = data_results_plot).facet(facet = 'Well', columns = plot_columns)

else:
    µ_max_point = alt.Chart(data_results_plot).mark_point(color = "red"
    ).encode(
        alt.X('Time:Q'),
        alt.Y('µ_max corresponding Value:Q')
    )
    
    plotted = alt.layer(raw_data, fitted_curves, tangent, µ_max_point, data = data_results_plot).facet(facet = 'Well', columns = plot_columns)

plotted.save("chart.svg", scale_factor = 5.0)
plotted


################################################################################
## Summarise results

if os.path.exists('result.md') == True:
    os.remove("result.md")

if os.path.exists('formulas.md') == True:
    os.remove("formulas.md")

#creates copy of dataframe 'data':
data_results = data.copy()

#checks if the new dataframe contains the columns 'µ_max' & 'y0' and deletes them if TRUE:
if hasattr(data_results, 'µ_max'):
    del data_results['µ_max']
    
    if hasattr(data_results, 'y0'):
        del data_results['y0']

#reorders the columns of the dataframe:
data_results_columns = list(data_results.columns)
new_columns_order = [data_results_columns[0], data_results_columns[-2], data_results_columns[-1]]
new_columns_order.extend(data_results_columns[1:-2])
data_results = data_results[new_columns_order]

#creates empty dictionary:
checkerboard_dict = {}

#get row-count of dataframe 'data_results':
row_count = data_results.shape[0]

#for-loop over all checkerboards:
for boards in range(len(checkerboards)):

    #initialize data for if-conditions:
    first_well = str(checkerboards[boards][1])
    last_well = str(checkerboards[boards][2])

    #creates empty array:
    well_list = []
    
    #for-loop over all wells in dataframe 'data_results':
    for rows in range(row_count):
        
        #get name of the specific well:
        well_name = str(data_results.iat[rows, 0])
        
        #check if well is within the limits of the checkerboard and accomplishs the log-level-reduction:
        if well_name[0] >= first_well[0] and well_name[0] <= last_well[0]:
            if well_name[1:] >= first_well[1:] and well_name[1:] <= last_well[1:]:

                #if TRUE append well to list:
                well_list.append(well_name)
        
        elif well_name[0] <= first_well[0] or well_name[0] >= last_well[0]:
            if well_name[1:] >= first_well[1:] and well_name[1:] <= last_well[1:]:
                
                #if TRUE append well to list:
                well_list.append(well_name)
        
        elif well_name[1:] <= first_well[1:] or well_name[1:] >= last_well[1:]:
            if well_name[0] >= first_well[0] or well_name[0] <= last_well[0]:
                
                #if TRUE append well to list:
                well_list.append(well_name)
    
    #creates copy of dataframe 'data_results':
    data_results_copy = data_results.copy()

    #gets actual index + 1:
    boards_index = (boards + 1)
    
    #checks if the index is < 10:
    if boards_index < 10:
        
        #if TRUE a 0 is added in front of the index and the so numbered Checkerboard is used as key in the dicitionary which references the dataframe 'data_results_copy' without all rows whose well wasn´t in well_list. Sets the column with the well-names as index:
        checkerboard_dict["Checkerboard_0{0}".format(boards_index)] = data_results_copy.drop(data_results_copy[data_results_copy.Well.isin(well_list) == False].index).set_index('Well', drop = True).to_markdown()
    
    elif boards_index >= 10:
        
        #if FALSE the  Checkerboard is just numbered with the index and used as key in the dicitionary which references the dataframe 'data_results_copy' without all rows whose well wasn´t in well_list. Sets the column with the well-names as index:
        checkerboard_dict["Checkerboard_{0}".format(boards_index)] = data_results_copy.drop(data_results_copy[data_results_copy.Well.isin(well_list) == False].index).set_index('Well', drop = True).to_markdown()

#for-loop over all checkerboards in the dictionary:
for keys in range(len(checkerboard_dict)):
    actual_key = list(checkerboard_dict)[keys]
    #print(actual_key, '\n', checkerboard_dict[actual_key], '\n')
    
    #variable that references actual checkerboard from the dictionary
    result_md = checkerboard_dict[actual_key]
    
    #creates a file 'result.md' with the command to append data to it:
    result_file = open("result.md", "a")
    
    #writes data to the opened file:
    result_file.write('## ')
    result_file.write(actual_key)
    result_file.write('\nThe average FICI-value of the Checkerboard is:  ')
    result_file.write(str(fici_average[boards]))
    result_file.write('\n')
    result_file.write(result_md)
    result_file.write('\n\n')
    
    #closes the file:
    result_file.close()

data_sigmoid_parameters = pd.DataFrame(popt_list, columns = ['Well',
    'Sigmoid-function parameters: L', 'Sigmoid-function parameters: x0', 
    'Sigmoid-function parameters: k', 'Sigmoid-function parameters: n'])

data_formulas = data.copy()

del data_formulas['SGT minus control']
del data_formulas['log-level-reduction']
del data_formulas['Nitroxolin-concentration']
del data_formulas['Dalbavancin-concentration']

if hasattr(data_formulas, 'Sigmoid-function parameters: L'):
    del data_formulas['Sigmoid-function parameters: L']
    
    if hasattr(data, 'Sigmoid-function parameters: x0'):
        del data['Sigmoid-function parameters: x0']
        
        if hasattr(data_formulas, 'Sigmoid-function parameters: k'):
            del data_formulas['Sigmoid-function parameters: k']
            
            if hasattr(data_formulas, 'Sigmoid-function parameters: n'):
                del data_formulas['Sigmoid-function parameters: n']
                data_formulas = data_formulas.merge(data_sigmoid_parameters, on = ['Well'])
            
            else:
                data_formulas = data_formulas.merge(data_sigmoid_parameters, on = ['Well'])
        else:
            data_formulas = data_formulas.merge(data_sigmoid_parameters, on = ['Well'])
    else:
        data_formulas = data_formulas.merge(data_sigmoid_parameters, on = ['Well'])

else:
    data_formulas = data_formulas.merge(data_sigmoid_parameters, on = ['Well'])

data_formulas = data_formulas.set_index('Well', drop = True)
data_formulas = data_formulas.rename(columns={'µ_max': 'm', 'y0': 'n'})

formulas_md = data_formulas.to_markdown()

sigmoid_formula = 'y = L / (1 + e^(-k * (x - x0))) + n'
tangent_formula = 'y = m * x + n'

result_file = open("formulas.md", "a")
result_file.write('## Parameters of the tangent- and the sigmoid-function of each well\n\n')
result_file.write('Parameters for the formulars of the tangent- and sigmoid-function of each well.\n')
result_file.write('The tangent-function is defined as:\n')
result_file.write(tangent_formula)
result_file.write('\n\n')
result_file.write('The sigmoid-function is defined as:\n')
result_file.write(sigmoid_formula)
result_file.write('\n\n')
result_file.write(formulas_md)
result_file.close()


from google.colab import files
files.download('result.md')
files.download('formulas.md')
files.download('chart.svg')

KeyboardInterrupt: ignored